In [3]:
import json
import pandas as pd
import numpy as np
import os
from os.path import join
from collections import Counter
from tqdm.notebook import tqdm

from disease_ontology import categorical_encoding, sorted_cancer_subtypes, map_histological_subtype, map_site_subtype

# COSMIC Data

For this implementation of the somatic-mutation-based embeddings of COSMIC samples we used the v92 version of the Mutation Data (https://cancer.sanger.ac.uk/cosmic/download).
The downloaded file is to be extracted in the same folder as this notebook.

Due to the size of the dataset, it will be opened and parsed in with a streaming configuration, which will lead to the creation of temporary files that can be deleted afterwards. The limitations due to the size of the dataset will require multiple passes over the data, especially for the frequency related quality requirements.

# Data Processing Choices

The raw data downloaded from the COSMIC website consists of a tab separated file in which each line represents a single somatic mutation in a sample. Each sample presents multiple mutations, and each tumor from a specific patient may be used to extract multiple samples. The model implemented in the other files is used to embed each sample based on the somatic mutations profile described in the dataset, which is a vector that corresponds to the available list of somatic mutations where a value of 1 in component $j$ of the vector signals that a mutation of gene $j$ is present in the sample, and where a 0 represents the absence of that mutation.

## Quality control

The somatic mutations in the COSMIC dataset are filtered according to the following criteria:
- labelled as "Confirmed somatic variant"
- they must come from a genome-wide screen study
- mapped onto the GRCh38 reference genome
- the mutations must correspond to a gene (only coding mutations)
- we exclude hypermutated samples (>1000 mutation) according to previous works in the literature (_Vogelstein, B., Papadopoulos, N., Velculescu, V. E., Zhou, S., Diaz, L. A., and Kinzler, K. W. (2013)._ Cancer genome landscapes)
- underrepresented genes are removed (mutations that appear fewer than 100 times)

In [ ]:
if not os.path.exists("tmp_data"):
    os.mkdir("tmp_data")
if not os.path.exists("data"):
    os.mkdir("data")

In [ ]:
df = pd.read_csv('CosmicGenomeScreensMutantExport.tsv', header=0, sep='\t', quotechar='"', error_bad_lines=False, chunksize=10000)

# First iteration through the data: filtering according to the qualitative requirements
with open("tmp_data/cosmic_filtered_tmp.csv", "w") as f:
    writer = csv.writer(f)
    for i, chunk in tqdm(enumerate(df)):
        chunk = chunk[(chunk["Mutation somatic status"] == "Confirmed somatic variant") & (chunk["Genome-wide screen"]== 'y')& 
                      (chunk["GRCh"]==38) & 
                      (~chunk["HGNC ID"].isna()) 
        chunk["HGNC ID"] = chunk["HGNC ID"].astype(int)
        chunk = chunk[["HGNC ID", "ID_sample", 'FATHMM prediction', 
                       "Primary site", 'Site subtype 1', 'Site subtype 2', 'Site subtype 3',
                       'Primary histology', 'Histology subtype 1', 'Histology subtype 2',
                       'Histology subtype 3']]

        if i==0:
            writer.writerow(chunk.columns.values)
        for r in chunk.values:
            writer.writerow(r)


In [ ]:
# Second pass over the dataset: remove hypermutated samples (>1000 mutations)
df = pd.read_csv('tmp_data/cosmic_filtered_tmp.csv', header=0, sep=',', quotechar='"', error_bad_lines=False, chunksize=100000)

sample_id_counter = Counter()
for i, chunk in tqdm(enumerate(df)):
    sample_id_counter.update(chunk["ID_sample"][~chunk["ID_sample"].isna()].values)
hypermutated_samples = [s for s, v in sample_id_counter.items() if v>1000]

df = pd.read_csv('tmp_data/cosmic_filtered_tmp.csv', header=0, sep=',', quotechar='"', error_bad_lines=False, chunksize=100000)
with open("tmp_data/cosmic_filtered_tmp2.csv", "w") as f:
    writer = csv.writer(f)
    for i, chunk in tqdm(enumerate(df)):
        chunk = chunk[(~chunk["ID_sample"].isin(hypermutated_samples))]        
        if i==0:
            writer.writerow(chunk.columns.values)
        for r in chunk.values:
            writer.writerow(r)

# Third pass over the dataset: remove underrepresented genes (<100 occurrences)
df = pd.read_csv('tmp_data/cosmic_filtered_tmp2.csv', header=0, sep=',', quotechar='"', error_bad_lines=False, chunksize=100000)
gene_id_counter = Counter()
for i, chunk in tqdm(enumerate(df)):
    gene_id_counter.update(chunk["HGNC ID"][~chunk["HGNC ID"].isna()].values) 
low_count_genes = [g for g, v in gene_id_counter.items() if v<100]

df = pd.read_csv('tmp_data/cosmic_filtered_tmp2.csv', header=0, sep=',', quotechar='"', error_bad_lines=False, chunksize=100000)
with open("tmp_data/cosmic_filtered.csv", "w") as f:
    writer = csv.writer(f)
    for i, chunk in tqdm(enumerate(df)):
        chunk = chunk[(~chunk["HGNC ID"].isin(low_count_genes))]        
        if i==0:
            writer.writerow(chunk.columns.values)
        for r in chunk.values:
            writer.writerow(r)


In [ ]:
# Splitting the filtered data into the relevant portions for the encoding of the mutations and the subtypes
# with this methods both can fit into memory without streaming

df = pd.read_csv('tmp_data/cosmic_filtered.csv', header=0, sep=',', quotechar='"', error_bad_lines=False, chunksize=100000)
with open("tmp_data/cosmic_filtered_subtype.csv", "w") as f:
    writer = csv.writer(f)
    for i, chunk in tqdm(enumerate(df)):
        
        chunk = chunk[["ID_sample", "Primary site", 'Site subtype 1', 'Site subtype 2', 'Site subtype 3',
       'Primary histology', 'Histology subtype 1', 'Histology subtype 2',
       'Histology subtype 3']]
        
        if i==0:
            writer.writerow(chunk.columns.values)
        for r in chunk.values:
            writer.writerow(r)
            
df = pd.read_csv('tmp_data/cosmic_filtered.csv', header=0, sep=',', quotechar='"', error_bad_lines=False, chunksize=100000)
with open("tmp_data/cosmic_filtered_mutation_data.csv", "w") as f:
    writer = csv.writer(f)
    for i, chunk in tqdm(enumerate(df)):
        
        chunk = chunk[["HGNC ID", "ID_sample", 'FATHMM prediction']]
        
        if i==0:
            writer.writerow(chunk.columns.values)
        for r in chunk.values:
            writer.writerow(r)

In [ ]:
# Encoding and saving each sample based on its site and histology subtype. The encoding is a binary vector where a 1 in component j represents that 
# the sample belongs to the subtype j, and 0 signals otherwise. The ordering of the subtypes is found in the disease_ontology module, and 
# the result of the processing is a numpy matrix where the ordering of the rows corresponds to the sorted ids found in "sample_ids.json"

df = pd.read_csv('tmp_data/cosmic_filtered_subtype.csv', header=0, sep=',', quotechar='"', error_bad_lines=False)
df = df.drop_duplicates(subset=['ID_sample'])

sample_ids = []
encodings = []
for i, (id_sample, row) in enumerate(zip(df.values[:,0], df.values[:,1:])):
    hist_subtype = map_histological_subtype(*row)
    site_subtype = map_site_subtype(*row)    
    encoding = np.array(categorical_encoding(site_subtype, hist_subtype))
    if np.sum(encoding)>0:
        if np.sum(encoding[-2:])>1:
            print(i)
        encodings.append(encoding)
        sample_ids.append(id_sample)
        
encodings = np.array(encodings)

with open("data/sample_ids.json", "w") as f:
    json.dump(sample_ids, f)
np.save("data/sample_encodings.npy", encodings)

In [ ]:
# Mapping the samples IDs to the corresponding mutated genes. Mutations set is a lookup table where each sample ID corresponds to two lists. The first includes
# the deleterious mutations, while the second groups non-deleterious mutations. The split is based on the FATHMM prediction included in the dataset.
# The file sorted_mutations.json gives a unique ordering for the mutated genes that will be used
# for the binary encoding of the mutation profile of each sample.

mutations_set = {s: ([], []) for s in sample_ids}
df = pd.read_csv('tmp_data/cosmic_filtered_mutation_data.csv', header=0, sep=',', quotechar='"', error_bad_lines=False)

for i, row in tqdm(df.iterrows()):
    if row["ID_sample"] in mutations_set.keys():
        if row["FATHMM prediction"] == "PATHOGENIC":
            mutations_set[row["ID_sample"]][0].append(int(row["HGNC ID"]))
        else:
            mutations_set[row["ID_sample"]][1].append(int(row["HGNC ID"]))
        
with open("data/mutations_mapping_split.json", "w") as f:
    json.dump(mutations_set, f, indent=2)
    
sorted_mutations = set()
for k, v in mutations_set.items():
    sorted_mutations.update(v[0])
    sorted_mutations.update(v[1])
    
sorted_mutations = sorted(list(sorted_mutations))
with open("data/sorted_mutations.json", "w") as f:
    json.dump(sorted_mutations, f, indent = 2)

In [4]:
# Deleting the temporary files
